<a href="https://colab.research.google.com/github/BrotherKim/KAIST_CODE/blob/master/SEP531/term/5_KoBERT_eval_ks_std_ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
SAVEPOINT_PATH = '/content/drive/MyDrive/KAIST/SEP531/KoBERT_emotion_finetuned_data_ks.pt'

In [2]:
# !ls -l /content/drive/MyDrive/KAIST/SEP531

#KoBERT 다운로드(사용)#

In [3]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-ryttht2m
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-ryttht2m
     |████████████████████████████████| 344 kB 5.0 MB/s 
     |████████████████████████████████| 46.9 MB 2.0 MB/s 
     |████████████████████████████████| 4.9 MB 57.7 MB/s 
     |████████████████████████████████| 1.2 MB 46.4 MB/s 
     |████████████████████████████████| 3.4 MB 53.6 MB/s 
     |████████████████████████████████| 596 kB 47.3 MB/s 
     |████████████████████████████████| 3.3 MB 49.6 MB/s 
     |████████████████████████████████| 61 kB 326 kB/s 
     |████████████████████████████████| 895 kB 14.9 MB/s 
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=13136 sha256=2ef2a9036bc468ce7b5ab9814c76f6cc5cc9539d5cd492b58651eac4564926be
  Stored in directory: /tmp/pip-ephem-wheel-cache-otc5mrwg/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd

In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [5]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [6]:
#GPU 사용
device = torch.device("cuda:0")

In [8]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

.cache/kobert_v1.zip[██████████████████████████████████████████████████]
.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


#실행 환경(사용)#

- Python >= 3.6
- PyTorch >= 1.70
- Transformers = 3.0.2
- Colab
- batch size = 64 (convertable)
- epochs = 10 (convertable)

In [9]:
!pip install -U torchtext==0.6.0
!pip install transformers

     |████████████████████████████████| 64 kB 2.4 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.11.0
    Uninstalling torchtext-0.11.0:
      Successfully uninstalled torchtext-0.11.0


In [10]:
#!pip install mxnet
#!pip install gluonnlp pandas tqdm
#!pip install sentencepiece
#!pip install transformers==3.0.2
#!pip install torch

# Libraries

import matplotlib.pyplot as plt
import pandas as pd
import torch

# Preliminaries

from torchtext.data import Field, TabularDataset, BucketIterator, Iterator

# Models

import torch.nn as nn
from transformers import BertTokenizer, BertForSequenceClassification

# Training

import torch.optim as optim

# Evaluation

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

In [11]:
#구글드라이브 연동
from google.colab import drive
drive._mount('/content/drive')

Mounted at /content/drive


#KoBERT 입력 데이터로 변환(사용)#

In [12]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [13]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

#추출한 데이터에 감정 레이블 추가(사용)#

In [14]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

새로운 문장을 테스트 할 때, 입력되는 문장을 KoBERT의 입력 형식으로 바꿔주는 코드를 작성해주어야 한다. 아래 코드를 작성하여 토큰화, 패딩, 텐서를 바꿔주고 예측을 하는 'predict' 함수를 만들어 주었다.

In [15]:
model1 = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
model1.load_state_dict(torch.load(SAVEPOINT_PATH), strict=False)
model1.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [16]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

dic = {0:'공포', 1:'놀람', 2:'분노', 3:'슬픔', 4:'중립', 5:'행복', 6:'혐오'}
dic = {0:'fear', 1:'surprise', 2:'anger', 3:'sadness', 4:'neutral', 5:'happiness', 6:'disgust'}
r_dic = {'fear':0, 'surprise':1, 'anger':2, 'sadness':3, 'neutral':4, 'happiness':5, 'disgust':6}
def getEmotion(out):
  idx_eval=[]
  val_eval=[]
  for i in out:
    logits=i
    logits = logits.detach().cpu().numpy()
    idx = np.argmax(logits)
    idx_eval.append(idx)
    val_eval.append(dic.get(idx))
    return idx_eval[0], val_eval[0]


def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model1.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model1(token_ids, valid_length, segment_ids)

        return getEmotion(out)


        # file wrting

using cached model


#Eval

In [17]:
filename = '/content/drive/MyDrive/KAIST/SEP531/std_test_small.csv'

In [38]:
import csv

texts = []
labels = []

with open(filename, 'r') as f:
  csv_data = csv.reader(f, delimiter='|')
  next(csv_data)
  for i, line in enumerate(csv_data):
    #if(i == 10000):
    #  break
    #print(int(line[1]))
    text = line[0]
    label = int(line[1])
    texts.append(text)
    labels.append(label)

In [39]:
labels

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [ ]:
texts = t[300:1300]

In [50]:

def Eval():
  for i in range(len(texts)):
    text = texts[i]
    label = labels[i]
    label_idx = label
    print(label_idx)

Eval()
  #print('>> Generated file %s/%s.answer\n' % (path, jsonfilename))

  #with open('%s/%s.data' % (path, jsonfilename), 'w') as f:
  #  for u in ut:
  #    f.write('%s\t%s\n' % (u['standard_form'], u['dialect_form']))


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4


In [24]:
y = []
p = []
def Eval():
  for i in range(len(texts)):
    text = texts[i]
    label = labels[i]
    label_idx = label
    #label_idx = r_dic.get(label)
    idx, val = predict(text)
    #print('(%s/%s) %s\n' % (val, label, text))
    #if(i % 1000 == 0):
    print('[%d](%c)(%d/%d) %s\n' % (i, 'O' if (idx == label_idx) else 'X', idx, label_idx, text))
    p.append(idx)
    y.append(label_idx)

  #print('>> Generated file %s/%s.answer\n' % (path, jsonfilename))

  #with open('%s/%s.data' % (path, jsonfilename), 'w') as f:
  #  for u in ut:
  #    f.write('%s\t%s\n' % (u['standard_form'], u['dialect_form']))


In [55]:
yy = y
pp = p

In [62]:
p = p[102:]
y = y[102:]

In [40]:
Eval()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[0](X)(4/0) 이제 결혼 아 애를 낳고 싶어 너는 자녀 계획은 어떻게 돼?

[1](O)(0/0) 나 같은 경우에는 여자아이랑 남자아이 둘 다 키워보고 싶어서

[2](X)(3/0) 그래도 셋 째까지는 도전해 볼 것 같애 만약에 둘 째에서 남자 하나 여자 하나 이렇게 나온다면은

[3](O)(0/0) 물론 거기서 멈추겠지만 만약 첫 째랑 둘 째랑 성별이 같다면은

[4](O)(0/0) 연애에는 연상이랑 하고 싶고 결혼은 연하랑 하고 싶고 이러면은 넌 어때?

[5](O)(0/0) 연하랑도 해보고 싶고 동갑이랑도 해 봐서 가장 나한테 잘 맞는 스타일이랑 결혼하고 싶은 케이스인 것 같애

[6](O)(0/0) 나 같은 경우에는 생각을 해봤을 때 우리가 이제 결혼할 때 쯤이면은

[7](O)(0/0) 직업이 생기고 직장에서 -아- 아직 초반이라 많이 바쁠 때 가는 거긴 하지만

[8](X)(3/0) 그래서 그 신혼 여행 이후에는 또 직장에 몰두해야 되니까

[9](X)(3/0) 어디 가나 사람이 많다 보니까 사람 많은 걸 별로 안 좋아하고

[10](X)(4/0) 그런 점이 유일한 단점이라고 생각해

[11](O)(0/0) 너는 데이트 통장 어떻게 생각해?

[12](O)(0/0) 이제 데이트 통장이라는 개념 자체가

[13](X)(3/0) 그 통장에 있는 돈으로 데이트 비용을 매번 계산하겠다는 개념이잖아.

[14](O)(0/0) 근데 그렇게 되면은 내 생각에는 너무 계산적이고 정 없고

[15](O)(0/0) 오히려 매달 뭐 이십 씩 채우기로 했는데 뭐 안 채우고 하면은 돈 쪽으로 너무 문제를 신경 쓸 것 같으니까

[16](O)(0/0) 그리고 애초에 얼마나 차이 난다고 약간 데이트 통장 같은 경우에는

[17](O)(0/0) 나 같은 경우에는 데이트 통장 많이 별로라고 생각해

[18](O)(0/0) 그러면은 결혼하고 나서는

[19](O)(0/0) 그럼 이제 통장 관리나 돈 관리 결혼하고 나서는 어떻게 하고 싶어?

[20](O)(0/0) 뭐 이런 

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f13b81d1290>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1295, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[7499](O)(4/4) 칼국수랑 거기는 버섯으로 육수를 내

[7500](X)(5/4) 칼국수랑 보쌈인데 속 나도 &company-name1& 되게 좋아하는데 &company-name1&랑 되게 다른 맛이야.

[7501](O)(4/4) 되게 다르게 맛있어서 비교를 할 수가 없는 게

[7502](O)(4/4) 그래 주소 찍어 줘 그러면은 음식중에서 한식 양식 일식 중식 뭐 좋아해?

[7503](O)(4/4) 난 다 좋아해.

[7504](O)(4/4) 그래도 그 중에 하나 있어?

[7505](O)(4/4) 그래도 한국인이다 보니깐 한식 좋아하기는 하는데 두 번째론 일식 좋아하지 나도 초밥 좋아하는데 여러 군데 있거든

[7506](O)(4/4) 요즘은 또 방어 철이다 보니까는 조만간 또 방어 먹겠네.

[7507](O)(4/4) 우리 작년 이브에 먹은 회가 뭐였지 광어 우럭 먹었나?

[7508](O)(4/4) 우리 그때 광어 &company-name7& 가서 어 광어 우리 같이 먹었었던 거 같앴었는데

[7509](O)(4/4) 방어를 한 번도 안 먹어 봤어?

[7510](O)(4/4) 어

[7511](O)(4/4) 오빠는 오빠는 근데 술을 안 좋아해서 다른가?

[7512](O)(4/4) 그 소주는 잘 안 먹지만 소맥은 그래도 어느 정도 먹으면 좋긴 하더라.

[7513](O)(4/4) 나는 근데 매운탕 때문에 술 먹는 것 같애 왜냐하면 술 먹고 매운탕 먹으면 뭔가 기분이 좋은 것 같아서

[7514](O)(4/4) 나는 먹을 걸 되게 좋아하는데 대전에는 막 맛집이 많진 않은거 같애 다 인스타 맛집 사진용 맛집 많은 거 같고

[7515](O)(4/4) &name3&는 매운 거 잘 먹잖아.

[7516](O)(4/4) 갑자기 막 뭐라 그러더라 &company-name5& 어 이런 데 닭발은 솔직히 너무 맵잖아.

[7517](O)(4/4) 한신 닭발도 백종원 거라 그러더라고 근데 

In [45]:
y

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


#F1 score

In [63]:
import sklearn.metrics as metrics
from sklearn.metrics import roc_auc_score

from sklearn.metrics import roc_auc_score

def roc_auc_score_multiclass(actual_class, pred_class, average = "macro"):

  #creating a set of all the unique classes using the actual class list
  unique_class = set(actual_class)
  roc_auc_dict = {}
  for per_class in unique_class:
    #creating a list of all the classes except the current class 
    other_class = [x for x in unique_class if x != per_class]

    #marking the current class as 1 and all other classes as 0
    new_actual_class = [0 if x in other_class else 1 for x in actual_class]
    new_pred_class = [0 if x in other_class else 1 for x in pred_class]

    #using the sklearn metrics method to calculate the roc_auc_score
    roc_auc = roc_auc_score(new_actual_class, new_pred_class, average = average)
    roc_auc_dict[per_class] = roc_auc

  return roc_auc_dict



In [64]:

print(metrics.precision_score(y, p, average=None)) #
print(metrics.precision_score(y, p, average=None).mean()) #
print(metrics.precision_score(y, p, average='macro')) #1.0
print(metrics.precision_score(y, p, average='micro')) #1.0

[0.69844358 0.80715851 0.69964664 0.7290401  0.79572732 0.76701031
 0.7839851 ]
0.7544302225732089
0.7544302225732089
0.7701770177017702


In [65]:
# sklearn 을 이용하면 전부 계산해준다.
print('accuracy', metrics.accuracy_score(y,p) )
print('precision1', metrics.precision_score(y, p, average=None)) #
print('precision2', metrics.precision_score(y, p, average=None).mean()) #
print('precision3', metrics.precision_score(y, p, average='macro')) #1.0
print('precision4', metrics.precision_score(y, p, average='micro')) #1.0
print('recall1', metrics.recall_score(y, p, average=None)) #
print('recall2', metrics.recall_score(y, p, average=None).mean()) #
print('recall3', metrics.recall_score(y, p, average='macro')) #1.0
print('recall4', metrics.recall_score(y, p, average='micro')) #1.0
print('f1_score1', metrics.f1_score(y, p, average=None)) #
print('f1_score2', metrics.f1_score(y, p, average=None).mean()) #
print('f1_score3', metrics.f1_score(y, p, average='macro')) #1.0
print('f1_score4', metrics.f1_score(y, p, average='micro')) #1.0

print(metrics.classification_report(y,p))
print(metrics.confusion_matrix(y,p))





accuracy 0.7701770177017702
precision1 [0.69844358 0.80715851 0.69964664 0.7290401  0.79572732 0.76701031
 0.7839851 ]
precision2 0.7544302225732089
precision3 0.7544302225732089
precision4 0.7701770177017702
recall1 [0.75341028 0.71061093 0.55307263 0.79155673 0.86126267 0.79317697
 0.50722892]
recall2 0.7100455890419684
recall3 0.7100455890419684
recall4 0.7701770177017702
f1_score1 [0.72488642 0.75581395 0.61778471 0.75901328 0.827199   0.77987421
 0.61594733]
f1_score2 0.7257884163194673
f1_score3 0.7257884163194673
f1_score4 0.7701770177017702
              precision    recall  f1-score   support

           0       0.70      0.75      0.72       953
           1       0.81      0.71      0.76      1555
           2       0.70      0.55      0.62       358
           3       0.73      0.79      0.76      1516
           4       0.80      0.86      0.83      3849
           5       0.77      0.79      0.78       938
           6       0.78      0.51      0.62       830

    accurac

In [67]:
print("\nLogistic Regression")
# assuming your already have a list of actual_class and predicted_class from the logistic regression classifier
lr_roc_auc_multiclass = roc_auc_score_multiclass(y, p)
print(lr_roc_auc_multiclass)

# Sample output
# Logistic Regression
# {0: 0.5087457159427196, 1: 0.5, 2: 0.5, 3: 0.5114706737345112, 4: 0.5192307692307693}
# 0.5078894317816


Logistic Regression
{0: 0.859570496510896, 1: 0.8396730645325892, 2: 0.7721280564442248, 3: 0.8694904942587333, 4: 0.8614443409437992, 5: 0.8841174564530525, 6: 0.7472887952727039}
